In [1]:
import os
import sys
sys.path.append('C:/ALL_FROM_DESKTOP/Data_Science_ENDtoEND proj/proj_1/src')


In [2]:
%pwd

'c:\\ALL_FROM_DESKTOP\\Data_Science_ENDtoEND proj\\proj_1\\research'

In [3]:
os.chdir('c:\\ALL_FROM_DESKTOP\\Data_Science_ENDtoEND proj\\proj_1'
)

In [4]:
%pwd

'c:\\ALL_FROM_DESKTOP\\Data_Science_ENDtoEND proj\\proj_1'

# **Entity/init.py**

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    train_data_path: Path
    test_data_path: Path
    evaluation_report_path: Path


# **Config/configuration.py**

In [6]:
from src.mathematicsScore.constants import *
from src.mathematicsScore.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            evaluation_report_path=config.evaluation_report_path
        )

        return model_evaluation_config

# **Components/model_evaluation.py**

In [8]:
import os
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.mathematicsScore.logging import logger
from src.mathematicsScore.config.configuration import ConfigurationManager


Config file path: C:\ALL_FROM_DESKTOP\Data_Science_ENDtoEND proj\proj_1\config\config.yaml
Params file path: C:\ALL_FROM_DESKTOP\Data_Science_ENDtoEND proj\proj_1\params.yaml


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        """
        Evaluate the trained model using test data.
        """
        try:
            # Load the trained model
            logger.info(f"Loading trained model from {self.config.model_path}")
            
            with open(self.config.model_path, 'rb') as f:
                model = pickle.load(f)
            
            logger.info("Model loaded successfully")

            # Load the transformed training data
            logger.info(f"Loading training data from {self.config.train_data_path}")
            train_data = pd.read_csv(self.config.train_data_path)
            logger.info(f"Training data loaded with shape: {train_data.shape}")

            # Load the transformed test data
            logger.info(f"Loading test data from {self.config.test_data_path}")
            test_data = pd.read_csv(self.config.test_data_path)
            logger.info(f"Test data loaded with shape: {test_data.shape}")

            # Determine target column
            target_column = 'math_score'
            if 'mathematics_score' in train_data.columns:
                target_column = 'mathematics_score'

            # Split features and target for training data
            X_train = train_data.drop(columns=[target_column])
            y_train = train_data[target_column]

            # Split features and target for test data
            X_test = test_data.drop(columns=[target_column])
            y_test = test_data[target_column]

            logger.info(f"Evaluation features shape: {X_test.shape}")
            logger.info(f"Evaluation target shape: {y_test.shape}")

            # Make predictions
            logger.info("Making predictions...")
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)

            # Calculate metrics
            logger.info("Calculating evaluation metrics...")
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
            
            train_mae = mean_absolute_error(y_train, y_train_pred)
            test_mae = mean_absolute_error(y_test, y_test_pred)
            
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)

            # Log metrics
            logger.info("=== Model Performance Metrics ===")
            logger.info(f"Training RMSE: {train_rmse:.4f}")
            logger.info(f"Test RMSE: {test_rmse:.4f}")
            logger.info(f"Training MAE: {train_mae:.4f}")
            logger.info(f"Test MAE: {test_mae:.4f}")
            logger.info(f"Training R²: {train_r2:.4f}")
            logger.info(f"Test R²: {test_r2:.4f}")

            # Save evaluation metrics
            metrics = {
                'train_rmse': float(train_rmse),
                'test_rmse': float(test_rmse),
                'train_mae': float(train_mae),
                'test_mae': float(test_mae),
                'train_r2': float(train_r2),
                'test_r2': float(test_r2),
                'target_column': target_column,
                'model_path': str(self.config.model_path)
            }

            # Save metrics as JSON
            import json
            with open(self.config.evaluation_report_path, 'w') as f:
                json.dump(metrics, f, indent=4)
            
            logger.info(f"Evaluation report saved to {self.config.evaluation_report_path}")
            logger.info("Model evaluation completed successfully")

            return metrics

        except Exception as e:
            logger.error(f"Error in model evaluation: {e}")
            raise e

# **Pipeline Test**

In [10]:
from src.mathematicsScore.config.configuration import ConfigurationManager
from src.mathematicsScore.logging import logger

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()
    logger.info(f"Model evaluation completed successfully")
    print("Evaluation Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value}")
except Exception as e:
    logger.error(f"Error occurred: {e}")
    raise e

Evaluation Metrics:
train_rmse: 5.323096836206968
test_rmse: 5.394377988346398
train_mae: 4.266645839367169
test_mae: 4.213968765249189
train_r2: 0.8743150325681966
test_r2: 0.8804162685332191
target_column: math_score
model_path: artifacts/model_trainer/model.pkl
